In [1]:
# epitools 패키지 설치 및 로드
install.packages("epitools")
install.packages("dplyr")
library(epitools)
library(parallel)
library(glue)
library(dplyr)

In [42]:
ctable_path <- "/home/hashjamm/results/disease_network/ctables/"
pids_info_path = "/home/hashjamm/results/disease_network/"

In [11]:
cut_ctable_1 <- read.csv(glue("{ctable_path}cut_ctable_1.csv"), header = TRUE)
cut_ctable_2 <- read.csv(glue("{ctable_path}cut_ctable_2.csv"), header = TRUE)
cut_ctable_3 <- read.csv(glue("{ctable_path}cut_ctable_3.csv"), header = TRUE)
cut_ctable_4 <- read.csv(glue("{ctable_path}cut_ctable_4.csv"), header = TRUE)
cut_ctable_5 <- read.csv(glue("{ctable_path}cut_ctable_5.csv"), header = TRUE)
cut_ctable_6 <- read.csv(glue("{ctable_path}cut_ctable_6.csv"), header = TRUE)
cut_ctable_7 <- read.csv(glue("{ctable_path}cut_ctable_7.csv"), header = TRUE)
cut_ctable_8 <- read.csv(glue("{ctable_path}cut_ctable_8.csv"), header = TRUE)
cut_ctable_9 <- read.csv(glue("{ctable_path}cut_ctable_9.csv"), header = TRUE)
cut_ctable_10 <- read.csv(glue("{ctable_path}cut_ctable_10.csv"), header = TRUE)

In [12]:
# 멀티 프로세싱

final_table_generator <- function(cut_ctable, filename, adjusted_method) {
    
    # 빈 숫자형 벡터 생성 (cut_ctable의 행 수에 맞게)
    n <- nrow(cut_ctable)
    rr_values <- vector("numeric", n)
    rr_lower_cis <- vector("numeric", n)
    rr_upper_cis <- vector("numeric", n)
    chisq_values <- vector("numeric", n)
    chisq_p_values <- vector("numeric", n)
    fisher_p_values <- vector("numeric", n)

    # 클러스터 생성
    cl <- makeCluster(detectCores())  # 사용 가능한 코어만큼 클러스터 생성

    # 클러스터에 필요한 데이터 및 패키지를 전송
    clusterExport(cl, varlist = c("cut_ctable", "riskratio"), envir = environment())  
    # 'cut_ctable'과 'riskratio' 함수 전달

    # 병렬 작업: parLapply 사용
    results <- parLapply(cl, 1:n, function(i) {
      row <- cut_ctable[i, ]

      # 2x2 교차표 데이터 생성 (각 행에 대한 2x2 테이블 생성)
      data <- matrix(rev(unlist(row[3:6])), nrow = 2, byrow = TRUE)

      # 상대 위험도 및 신뢰구간 계산
      rr_result <- riskratio(data)$measure[2, ]
      rr_value <- rr_result[1]
      rr_lower_ci <- rr_result[2]
      rr_upper_ci <- rr_result[3]

      # 카이제곱 검정
      chisq_result <- chisq.test(data)
      chisq_value <- chisq_result[[1]]
      chisq_p_value <- chisq_result[[3]]

      # 피셔의 정확 검정
      fisher_result <- fisher.test(data)
      fisher_p_value <- fisher_result[[1]]

      # 결과를 벡터로 반환
      return(c(rr_value, rr_lower_ci, rr_upper_ci, chisq_value, chisq_p_value, fisher_p_value))
    })

    # 병렬 처리 결과를 매트릭스로 변환
    results <- do.call(rbind, results)

    # 클러스터 종료
    stopCluster(cl)

    # 매트릭스에서 각 열 추출하여 cut_ctable에 붙임
    cut_ctable['rr_values'] = results[, 1]
    cut_ctable['rr_lower_cis'] = results[, 2]
    cut_ctable['rr_upper_cis'] = results[, 3]
    cut_ctable['log_rr_values'] <- log1p(cut_ctable$rr_values) # 🔹 로그 변환된 RR 추가
    cut_ctable['chisq_values'] = results[, 4]
    cut_ctable['chisq_p_values'] = results[, 5]
    cut_ctable['fisher_p_values'] = results[, 6]
    
    # Bonferroni 보정된 p-values 추가
    cut_ctable['adjusted_chisq_p_values'] <- p.adjust(cut_ctable$chisq_p_values, method = adjusted_method)
    cut_ctable['adjusted_fisher_p_values'] <- p.adjust(cut_ctable$fisher_p_values, method = adjusted_method)
    
    # rr value에 대하여 내림차순 정렬
    cut_ctable <- cut_ctable %>% arrange(desc(rr_values))
    
    # 테이블 저장
    file_path <- file.path(ctable_path, filename)
    write.csv(cut_ctable, file = file_path, row.names = FALSE)
    
    }

In [13]:
final_table_generator(cut_ctable_1, "stat_cut_result_1.csv", "fdr")
final_table_generator(cut_ctable_2, "stat_cut_result_2.csv", "fdr")
final_table_generator(cut_ctable_3, "stat_cut_result_3.csv", "fdr")
final_table_generator(cut_ctable_4, "stat_cut_result_4.csv", "fdr")
final_table_generator(cut_ctable_5, "stat_cut_result_5.csv", "fdr")
final_table_generator(cut_ctable_6, "stat_cut_result_6.csv", "fdr")
final_table_generator(cut_ctable_7, "stat_cut_result_7.csv", "fdr")
final_table_generator(cut_ctable_8, "stat_cut_result_8.csv", "fdr")
final_table_generator(cut_ctable_9, "stat_cut_result_9.csv", "fdr")
final_table_generator(cut_ctable_10, "stat_cut_result_10.csv", "fdr")

In [25]:
# stat_cut_result_1 <- read.csv(glue("{ctable_path}stat_cut_result_1.csv"), header = TRUE)
# stat_cut_result_2 <- read.csv(glue("{ctable_path}stat_cut_result_2.csv"), header = TRUE)
# stat_cut_result_3 <- read.csv(glue("{ctable_path}stat_cut_result_3.csv"), header = TRUE)
# stat_cut_result_4 <- read.csv(glue("{ctable_path}stat_cut_result_4.csv"), header = TRUE)
# stat_cut_result_5 <- read.csv(glue("{ctable_path}stat_cut_result_5.csv"), header = TRUE)
# stat_cut_result_6 <- read.csv(glue("{ctable_path}stat_cut_result_6.csv"), header = TRUE)
# stat_cut_result_7 <- read.csv(glue("{ctable_path}stat_cut_result_7.csv"), header = TRUE)
# stat_cut_result_8 <- read.csv(glue("{ctable_path}stat_cut_result_8.csv"), header = TRUE)
# stat_cut_result_9 <- read.csv(glue("{ctable_path}stat_cut_result_9.csv"), header = TRUE)
# stat_cut_result_10 <- read.csv(glue("{ctable_path}stat_cut_result_10.csv"), header = TRUE)

In [36]:
# stat_cut_result_1 %>% head()

,cause_abb,outcome_abb,ct00,ct01,ct10,ct11,rr_values,rr_lower_cis,rr_upper_cis,chisq_values,chisq_p_values,fisher_p_values,adjusted_chisq_p_values,adjusted_fisher_p_values
,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,N96,Z34,119,11,7,19,3.400000,1.8010179,6.418592,54.154286,1.853492e-13,1.662959e-11,4.989345e-12,4.193596e-10
2,H28,E11,410,45,30,61,2.733333,2.0359630,3.669571,154.643027,1.675729e-35,1.739195e-29,1.548132e-33,1.477508e-27
3,H36,E11,7037,508,564,945,2.495390,2.3369677,2.664552,2911.542953,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,E10,E11,5940,420,559,713,2.125224,1.9966473,2.262080,2046.464767,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,O22,J03,53,12,6,7,1.766667,0.9710835,3.214050,5.566459,1.830775e-02,1.221760e-02,6.349164e-02,4.668154e-02
6,J46,J45,2685,270,304,287,1.766447,1.6319978,1.911973,575.174830,4.203983e-127,1.724613e-101,2.829136e-124,9.783615e-99
